In [ ]:
import pickle
from transformers import AutoTokenizer

fixed_preds = pickle.load(open("fixed_preds.pkl", "rb"))
examples = pickle.load(open("fixed_preds_examples.pkl", "rb"))

tokenizer = AutoTokenizer.from_pretrained("/ML-A100/team/mm/zhangge/gezhangmv/SKGLM/models/uskg-multitask")

# use multiple workers  to launch 16 process for tokenizer.batch_decode of fixed_preds
from multiprocessing import Pool
from functools import partial
import tqdm

def batch_decode(preds, tokenizer, num_proc=16):
    with Pool(num_proc) as p:
        return list(tqdm.tqdm(p.imap(partial(tokenizer.decode, skip_special_tokens=True), preds), total=len(preds)))
    
fixed_preds_decoded = batch_decode(fixed_preds, tokenizer)

import json
predictions = fixed_preds_decoded
with open(f"output/v13_uskg_inst_test/predictions_predict.json", "w") as f:
                json.dump(
                    [dict(**{"prediction": predictions[idx]}, **examples[idx]) for idx in range(len(predictions))],
                    f,
                    indent=4,
                )